In [16]:
from model import AttentionModel

In [10]:
keyPoints = {
    'NOSE': 0,
    'LEFT_EYE': 1,
    'RIGHT_EYE': 2,
    'LEFT_EAR':3,
    'RIGHT_EAR': 4,
    'LEFT_SHOULDER': 5,
    'RIGHT_SHOULDER':6,
    'LEFT_ELBOW': 7,
    'RIGHT_ELBOW': 8,
    'LEFT_WRIST': 9,
    'RIGHT_WRIST': 10,
    'LEFT_HIP': 11,
    'RIGHT_HIP': 12,
    'LEFT_KNEE': 13,
    'RIGHT_KNEE': 14,
    'LEFT_ANKLE': 15,
    'RIGHT_ANKLE': 16,
}

In [17]:
MODEL_SLUG = '60_yolov8m_best'

In [18]:
OUT_PATH = f'result/model_output_video_{MODEL_SLUG}.avi' # your video path for saving
DATA_PATH = 'Анализ бригад (телефон)/Есть телефон/00_26_30.mp4' # your video path for processing
SAVE_IMG_PATH = 'result'

In [19]:
model = AttentionModel(f'models/{MODEL_SLUG}.pt')
model.load_models()

In [20]:
model.detected_data

,Frame_id,Time,Position


In [25]:
model.process_video(DATA_PATH, OUT_PATH, pos_estimation=True, detection=True, save=True)

100%|██████████| 5196/5196 [07:53<00:00, 10.97it/s]

Time: 473.8496880531311


In [41]:
dropeddf  = model.pos_est_data.drop_duplicates(subset=['Frame_id', 'Time', "People_count"])

In [42]:
len(dropeddf)

5196

In [43]:
dropeddf.head(10)

,Frame_id,Time,People_count,KeyPoints
0,1,0.083333,2,"[[[1002.46533203125, 663.6934814453125], [989...."
1,2,0.166667,2,"[[[1013.07177734375, 678.38623046875], [996.08..."
2,3,0.250000,2,"[[[1033.189697265625, 691.5792236328125], [101..."
3,4,0.333333,2,"[[[1020.4559326171875, 678.0460205078125], [10..."
4,5,0.416667,2,"[[[1015.8204345703125, 672.3037719726562], [99..."
5,6,0.500000,2,"[[[1021.3914794921875, 670.3060302734375], [10..."
6,7,0.583333,2,"[[[1022.5838623046875, 673.1045532226562], [10..."
7,8,0.666667,2,"[[[1022.0577392578125, 676.2551879882812], [10..."
8,9,0.750000,2,"[[[1028.499267578125, 688.089599609375], [1007..."
9,10,0.833333,3,"[[[1028.941650390625, 692.9158935546875], [100..."


In [48]:
model.detected_data.tail()

,Frame_id,Time,Position
1918,5075,422.916667,"{'cell phones': [[411.7886, 158.0409, 514.842,..."
1919,5107,425.583333,"{'cell phones': [[414.74863, 155.55887, 514.50..."
1920,5160,430.000000,"{'cell phones': [[1877.963, 420.0927, 1917.593..."
1921,5165,430.416667,"{'cell phones': [[1880.9401, 418.65982, 1917.8..."
1922,5166,430.500000,"{'cell phones': [[1879.9912, 418.1053, 1917.86..."


In [49]:
yolodf = model.detected_data

In [51]:
yolodf = yolodf.set_index('Frame_id')

In [52]:
yolodf.head()

,Time,Position
Frame_id,,
1,0.083333,"{'cell phones': [[353.6478, 334.2564, 402.1416..."
2,0.166667,"{'cell phones': [[353.80054, 334.19482, 402.31..."
3,0.250000,"{'cell phones': [[353.91556, 334.04474, 402.43..."
4,0.333333,"{'cell phones': [[353.88528, 333.9823, 402.315..."
5,0.416667,"{'cell phones': [[353.71866, 334.07675, 402.18..."


In [54]:
dropeddf = dropeddf.set_index('Frame_id')


KeyError: "None of ['Frame_id'] are in the columns"

In [55]:
dropeddf.head()

,Time,People_count,KeyPoints
Frame_id,,,
1,0.083333,2,"[[[1002.46533203125, 663.6934814453125], [989...."
2,0.166667,2,"[[[1013.07177734375, 678.38623046875], [996.08..."
3,0.250000,2,"[[[1033.189697265625, 691.5792236328125], [101..."
4,0.333333,2,"[[[1020.4559326171875, 678.0460205078125], [10..."
5,0.416667,2,"[[[1015.8204345703125, 672.3037719726562], [99..."


In [71]:
[bbox['cell phones'] for bbox in yolodf.loc[1, 'Position'].values]

[[array([     353.65,      334.26,      402.14,       424.4], dtype=float32)],
 [array([     353.65,      334.26,      402.14,       424.4], dtype=float32)],
 [array([     353.65,      334.26,      402.14,       424.4], dtype=float32)]]

In [75]:
wrists = [pose[9:11] for pose in dropeddf.loc[1, 'KeyPoints']]
wrists

[[[1011.9351196289062, 661.543212890625],
  [1397.3941650390625, 704.1448974609375]],
 [[311.62347412109375, 522.0468139648438],
  [287.2590026855469, 418.62066650390625]]]

In [76]:
def get_center(bbox):
    return [(bbox[0] + bbox[2]) / 2, (bbox[1] + bbox[3]) / 2]

In [57]:
history = []
for frame_id in dropeddf.index:
    if frame_id in yolodf.index:
        phones = [bbox['cell phones'] for bbox in yolodf.loc[frame_id, 'Position'].values]
        wrists = [pose[9:11] for pose in dropeddf.loc[frame_id, 'KeyPoints']]
        # iter_arr = history if len(history) < len(phones) else phones
        if len(history) < len(phones):
            for bbox in history:
                bbox_center = get_center(bbox)

                bbox[]
            

SyntaxError: incomplete input (1692528271.py, line 5)

In [46]:
import numpy as np

(dropeddf.Frame_id.values != np.arange(1, len(dropeddf.Frame_id.values) + 1)).sum()

0

In [77]:
dropeddf.to_csv('posest.csv')

In [78]:
yolodf.to_csv('yolo.csv')

In [44]:
dropeddf.tail(10)

,Frame_id,Time,People_count,KeyPoints
5246,5187,432.250000,2,"[[[1259.4752197265625, 593.263671875], [1246.9..."
5247,5188,432.333333,2,"[[[1272.059326171875, 600.8618774414062], [126..."
5248,5189,432.416667,2,"[[[1281.7352294921875, 604.2354736328125], [12..."
5249,5190,432.500000,2,"[[[1296.75341796875, 604.5177612304688], [1282..."
5250,5191,432.583333,2,"[[[1300.808837890625, 605.81005859375], [1286...."
5251,5192,432.666667,2,"[[[1289.404052734375, 608.2304077148438], [127..."
5252,5193,432.750000,2,"[[[1293.4332275390625, 593.5037231445312], [12..."
5253,5194,432.833333,2,"[[[1293.38916015625, 589.206298828125], [1283...."
5254,5195,432.916667,2,"[[[1283.82666015625, 588.625244140625], [1274...."
5255,5196,433.000000,2,"[[[1294.331787109375, 575.75146484375], [1283...."


In [38]:
(dropeddf.index.values != dropeddf['Frame_id'].values - 1).sum()

5180

In [ ]:
import pandas as pd

pos_est_data = pd.DataFrame(model.pos_est_values, columns=['Frame_id', 'Time', *model.keypoints])
pos_est_data

In [ ]:
pos_est_data

In [ ]:
model.pos_est_data.head(10)

In [ ]:
model.pos_est_data.info()

In [ ]:
from ultralytics import YOLO

model = YOLO('models/yolov8x-pose.pt').to('cuda')
result = model.predict(DATA_PATH, stream=True, save=True)

In [ ]:
model('result/model_output_video.avi')

In [ ]:
model.pos_est_data.to_csv('result/pos_est_data.csv', index=0)

In [ ]:
model.detected_data.to_csv('result/detected_data.csv', index=0)